In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set()
from matplotlib import style
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
# plt.style.use('fivethirtyeight')
#plt.style.use(['ggplot'])
%config IPCompleter.greedy=True
#plt.style.use(['dark_background'])
plt.style.use('seaborn-dark')
#import json
import unicodedata

In [116]:
FR_csv = pd.read_csv("FRvideos.csv", encoding='latin1', skipinitialspace=True, engine='python')
#remove whitespace

FR_csv

In [64]:
FR_csv.fillna(0, inplace=True)

In [65]:
FR_csv.describe()

,category_id,views,likes,dislikes,comment_count
count,28669.000000,2.866900e+04,2.866900e+04,2.866900e+04,28669.000000
mean,20.077959,3.761308e+05,1.598380e+04,7.638781e+02,1622.871359
std,7.010469,1.679915e+06,7.570963e+04,1.281039e+04,12430.417101
min,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,17.000000,1.288000e+04,2.590000e+02,1.400000e+01,45.000000
50%,23.000000,5.910300e+04,1.582000e+03,6.800000e+01,201.000000
75%,24.000000,2.299520e+05,7.422000e+03,2.920000e+02,761.000000
max,43.000000,1.009116e+08,2.656682e+06,1.353661e+06,827755.000000


In [66]:
FR_csv.corr()

,category_id,views,likes,dislikes,comment_count
category_id,1.000000,-0.036631,-0.076275,-0.007088,-0.031473
views,-0.036631,1.000000,0.557804,0.570376,0.541414
likes,-0.076275,0.557804,1.000000,0.494787,0.804808
dislikes,-0.007088,0.570376,0.494787,1.000000,0.789565
comment_count,-0.031473,0.541414,0.804808,0.789565,1.000000


In [67]:
FR_csv.dtypes

video_id                   object
trending_date              object
title                      object
channel_title              object
category_id                 int64
publish_time               object
tags                       object
views                     float64
likes                     float64
dislikes                  float64
comment_count             float64
thumbnail_link             object
comments_disabled          object
ratings_disabled           object
video_error_or_removed     object
description                object
dtype: object

In [68]:
FR_csv['views'].isnull().values.any()

False

In [69]:
FR_csv['likes'].isnull().values.any()

False

In [70]:
FR_csv['dislikes'].isnull().values.any()

False

In [71]:
FR_csv['comment_count'].isnull().values.any()

False

#### Convert NaN values to 0

In [72]:
FR_csv['comment_count'] = FR_csv[['comment_count']].replace(np.NaN, 0)

In [73]:
FR_csv['likes'] = FR_csv[['likes']].replace(np.NaN, 0)

In [74]:
FR_csv['dislikes'] = FR_csv[['dislikes']].replace(np.NaN, 0)

In [75]:
FR_csv['comment_count'] = FR_csv[['comment_count']].replace(np.NaN, 0)

In [76]:
FR_csv['comment_count'].isnull().values.any()

False

In [77]:
FR_csv.dtypes

video_id                   object
trending_date              object
title                      object
channel_title              object
category_id                 int64
publish_time               object
tags                       object
views                     float64
likes                     float64
dislikes                  float64
comment_count             float64
thumbnail_link             object
comments_disabled          object
ratings_disabled           object
video_error_or_removed     object
description                object
dtype: object

In [78]:
#Finding NaN values
print(FR_csv[FR_csv['views'].isnull()])

Empty DataFrame
Columns: [video_id, trending_date, title, channel_title, category_id, publish_time, tags, views, likes, dislikes, comment_count, thumbnail_link, comments_disabled, ratings_disabled, video_error_or_removed, description]
Index: []


In [81]:
FR_csv[['likes','dislikes','views']] = FR_csv[['likes','dislikes','views']].astype(int)

In [82]:
FR_csv.dtypes

video_id                   object
trending_date              object
title                      object
channel_title              object
category_id                 int64
publish_time               object
tags                       object
views                       int64
likes                       int64
dislikes                    int64
comment_count             float64
thumbnail_link             object
comments_disabled          object
ratings_disabled           object
video_error_or_removed     object
description                object
dtype: object

### Understanding

In [83]:
FR_csv.select_dtypes(['int']).describe().T

,count,mean,std,min,25%,50%,75%,max
category_id,28669.0,20.077959,7.010469e+00,1.0,17.0,23.0,24.0,43.0
views,28669.0,376130.751753,1.679915e+06,0.0,12880.0,59103.0,229952.0,100911567.0
likes,28669.0,15983.802121,7.570963e+04,0.0,259.0,1582.0,7422.0,2656682.0
dislikes,28669.0,763.878126,1.281039e+04,0.0,14.0,68.0,292.0,1353661.0


In [109]:
FR_csv.replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]', value=r'')
# = df['A1'].astype(float64)

### Plot correlations between continuous features

In [118]:
# Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html
FR_csv = pd.read_csv("FRvideos.csv")
sns.set(style="white")

# Select columns containing continuous data
continuous_columns = FR_csv[['views','likes','dislikes','comment_count']].columns

# Calculate correlation of all pairs of continuous features
corr = FR_csv[continuous_columns].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom colormap - blue and red
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,
            square=True, xticklabels=True, yticklabels=True,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
plt.yticks(rotation = 0)
plt.xticks(rotation = 45)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd9 in position 254: unexpected end of data